In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset
import tqdm

In [2]:
model_path = 'data/Meta-Llama-3-8B-Instruct_Full_Patent2_QA3'
quant_path = 'data/Meta-Llama-3-8B-Instruct_Full_Patent2_QA3_awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
text_path ="./data/LLaMA-Factory/LLaMA-Factory/data/ai_kr_small.txt"

In [4]:
def tokenize_string(text, num_tokens=400):
    tokens = tokenizer.encode(text)
    cut = tokenizer.decode(tokens[:num_tokens])
    return cut

In [5]:
def load_data():
    lines = []
    with open(text_path, 'r', encoding='utf-8') as file:
        for i,line in enumerate(file):
            lines.append(tokenize_string(line[:4000]).strip().replace("<|begin_of_text|>",""))
            if i ==5000:
                break
    return lines

In [6]:
c = load_data()

In [7]:
c[0]

'skey:3519324000543 Patent Number:10-2017-0173498 Title:Air-Cooling type Adsorption Module Cooling System and Method Abstract:The present invention relates to an air-cooled adsorption module cooling system and a method thereof and, more specifically, to an air-cooled adsorption module cooling system and a method thereof capable of cooling an adsorption module and target gas injected into the adsorption module with cold air and adjusting adsorption and desorption by controlling the temperature. According to an embodiment of the present invention, the air-cooled adsorption module cooling system includes: a cooling unit which includes an adsorption module and a gas cooling coil, and cools target gas with cooled air; a blower which injects air into the cooling unit; a cooling coil which cools air injected into the cooling unit; a compression unit which lowers the temperature of the cooling coil; a heater which adjusts the temperature of the air injected into the cooling unit; and a control

In [8]:
# Quantize
model.quantize(tokenizer, quant_config=quant_config, calib_data=c)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

AWQ: 100%|██████████| 32/32 [11:16<00:00, 21.14s/it]

[2024-06-14 07:09:21,654] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Model is quantized and saved at "data/Meta-Llama-3-8B-Instruct_Full_Patent2_QA3_awq"
